In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("insurance_claims.csv")

In [ ]:
df.head(1000)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
(df.isnull().sum() / len(df)) * 100

In [ ]:
df.isnull().any(axis=1).sum() # Number of rows with at least 1 missing value

In [ ]:
df.isnull().all(axis=1).sum() # Number of rows with all values missing

In [ ]:
df = df.drop(columns=["_c39"])

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
df

# Insurance Fraud Feature Explanations
#### months_as_customer:
- Only 1 row contains a 0, could possibly mean that customer has been a customer for less than a month. Most likely fine to ignore

#### age:
- No rows contain 0, could possibly be used as a correlation

#### policy_number:
- This set of data is not important and can be dropped

#### policy_bind_date:
- AKA. binding date. Refers to "the date when an insurance policy officially becomes active - i.e., when the insurer commits ("binds") to providing coverage under the agreed terms. Or the start date of the policy's legal effect.
- We can convert the date into a datetime object in the df, and then extract, day, month, year. Could find trends where fraud happens more frequently relative to time of the year. Whether older or newer policies have more fraud. 

#### policy_state:
- Could be relevant, as to which state has more fraud and vice versa

#### policy_csl:
- Represents dollar amount of coverage (liability limit) chosen by policyholder. Example: 100/300 -> `$100,000` per person, `$300,000` per accident. Low CSL (small coverage limit) may indicate cost-conscious customers (not necessarily fraud, but often low payouts). Very high CSL could mean person wanted a big payout potential, fraudsters might intentionally choose high coverage to get larger claims.

#### policy_deductible:
- The amount the policyholder has to pay out of pocket before the insurance company starts paying for a claim. e.g. 1000 is `$1000` a customer has to pay out of pocket, and lets say a car repair costs `$3000`, the insurance pays `$2000`.
- A low deductible means pay less out of pocket and therefore have a higher monthly premium (pays more every month) and vice versa.
- Fraudsters may choose low deductibles to get larger payouts and larger deductibles suggests low-risk, cost-conscious customer which often leads to less fraud commitment.
- If the deductible doesn't match income or policy type, then its suspicious (e.g. low-income customer w/ expensive policy and low deductible)

#### policy_annual_premium:
- Total amount the policyholder pays per year to keep their insurance policy active. Based on many factors like policy_csl, policy_deductible, the insured's risk profile (age, driving history, etc.), car type and location, past claim history. For high premiums, fraudsters may target big payouts.
- For low premiums, could mean minimal coverage or a cheap policy fraudsters use to test the system. A mismatch with coverage and deductible could indicate fraud, like if someone pays a low premium has a high coverage limit.

#### umbrella_limit:
- Umbrella policy refers to extra liability coverage that goes beyond limits of someone's regular insurance policy (auto).
- Umbrella limit is the maximum amount of extra coverage (in dollars) provided by that umbrella policy (backup insurance for insurance)
- 0 means no umbrella policy, which could indicate a regular customer that may not have much to lose
- 1,000,000+ is high coverage for wealthier or more cautious individuals, could also attract fraudsters aiming for larger payouts

#### insured_zip:
- Refers to a postal code
- This is more categorical (geographical)
- Up to us if we want to include this data in

#### insured_sex:
- Male or female
- Up to us if we want to include this data in
- Let's try not to be sterotypical

#### insured_education_level:
- Education level of each policyholder
- Could be relevant by level, the higher education level the least likely to commit insurance fraud and vice versa.
- Up to use if we want to include this data in.

#### insured_occupation:
- Occupation of each policyholder
- Could be widely inconsistent as there are only 1000 cases and tons of different occupations, there may not be a pattern to be found

#### insured_hobbies:
- Hobby of each policyholder
- Could be widely inconsistent as there are only 1000 cases and tons of different hobbies, there may not be a pattern to be found

#### insured_relationship:
- Married or not married, child, relative, etc.
- Good to point out that husband and wife categories should be consolidated to just "married" if removing insured_sex from consideration

#### capital_gains:
- Shows how much money the insured person earned from investments (not from job/salary)
- Correlates with wealth, financial literacy, economic stability
- People under financial stress may be more tempted to commit fraud
- capital_gains can act as proxy for how financially stable someone is.

#### capital_loss:
- Shows how much money the person lost from investments like stocks, bonds, property
- Correlates with wealth, financial literacy, economic stability
- People with some capital loss are likely active investors
- People with no capital gains/losses might not invest at all
- People with large losses might be under financial stress
- capital_loss can act as proxy for how finanically literate/stable someone is
- High gains/low loss -> low fraud likelihood
- Low gain/high loss -> possibly higher fraud likelihood
- Both zero -> neutral baseline

#### incident_date:
- Date the incident happened
- Shorter times between incident and insured date could indicate fraud (preplanned accident)
- Buying insurance soon after the incident could indicate fraud

#### incident_type:
- Could find a pattern between incident types and fraudulent cases

#### collision_type:
- Type of collision could indicate fraud
- Some cases include ? because some incident types are related to parking or vehicle theft
- Replace these ? with NA

#### incident_severity:
- Indicative of the damage done per incident

#### authorities_contacted:
- Important to note Other could refer to tow truck companies, insurance agent, roadside assitance, or even a family/friend

#### incidient_state
- Possible pattern of which states have frequent fraudlent cases

#### incident_city
- Possible pattern of which cities have frequent fraudlent cases

#### incident_location	
- Address of location incident, most likely not helpful information
- Can remove entirely

#### incident_hour_of_the_day	
- Possible pattern of what times have more frequent fraudulent cases

#### number_of_vehicles_involved	
- Single vehicle incidents often easier to fake
- Double vehicle incidents not as easy but could still potentially indicate some cases of fraud
- Any more is harder to prove fraud

#### property_damage
- Either yes or no property damage
- ? might indicate that property damage wasn't reported at all
- There seems to not be a consistent pattern between property_damage and incident_type

#### bodily_injuries	
- Numerical data
- Lower numbers could indicate potential fraud if the incident was purposeful

#### witnesses	
- Lower count of witnesses might indicate more fraudulent cases

#### police_report_available	
- Categorical data with Yes or No
- ? Could indicate if information was not given on whether or not the police report is available (or somehow it got lost...)

#### total_claim_amount:
- Total dollar amount the insured person is claiming from the insurance company for a given incident.
- Unusually high claims could indicate fraud
- Fraud can be seen via other features like incident_type (large amount of claim and minor accidnt)
- Inconsistent data may not always lead to fraud, could be mismatch or underreporting

#### injury_claim:	
- Part of the total claim
  
#### property_claim:
- Part of the total claim
  
#### vehicle_claim:
- Part of the total claim
  
#### auto_make	
- Car make
- Could potentially be useful for fraud detection model, but too many makes that no real pattern may emerge
- Best to remove all together
  
#### auto_model	
- Car model
- Could potentially be useful for fraud detection model, but too many model's that no real pattern may emerge
- Best to remove all together
  
#### auto_year	
- Car year
- Could potentially be useful for fraud detection model, but too many model's that no real pattern may emerge
- Best to remove all together
#### fraud_reported
- Output label, Yes or No

In [ ]:
df = df.dropna(subset=['fraud_reported'])
numeric_cols = df.select_dtypes(include=['int64','float64']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())
cat_cols = df.select_dtypes(include='object').columns
df[cat_cols] = df[cat_cols].fillna('Unknown')

In [ ]:
df['policy_bind_date'] = pd.to_datetime(df['policy_bind_date'])
df['incident_date'] = pd.to_datetime(df['incident_date'])
df['policy_bind_year'] = df['policy_bind_date'].dt.year
df['incident_year'] = df['incident_date'].dt.year
df = df.drop(columns=['policy_bind_date', 'incident_date'])

In [ ]:
df = df.replace("?", np.nan)

In [ ]:
df.to_csv("insurance_claims_cleaned.csv", index=False)

In [ ]:
df.head()